In [2]:
# Install the following packages in the environment
%pip install numpy
%pip install matplotlib
%pip install pandas
%pip install scikit-learn

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

train_data = pd.read_csv('data/train.csv', header=None)
train_label_data = pd.read_csv('data/trainLabels.csv', header=None)

X = train_data
y = train_label_data


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [27]:
# See if there are any missing values
missing_val_count_by_column = (X.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


In [26]:
# Explore the data
print(X.head())

         0         1         2         3         4         5         6   \
0  0.299403 -1.226624  1.498425 -1.176150  5.289853  0.208297  2.404498   
1 -1.174176  0.332157  0.949919 -1.285328  2.199061 -0.151268 -0.427039   
2  1.192222 -0.414371  0.067054 -2.233568  3.658881  0.089007  0.203439   
3  1.573270 -0.580318 -0.866332 -0.603812  3.125716  0.870321 -0.161992   
4 -0.613071 -0.644204  1.112558 -0.032397  3.490142 -0.011935  1.443521   

         7         8         9   ...        30        31        32        33  \
0  1.594506 -0.051608  0.663234  ... -0.850465 -0.622990 -1.833057  0.293024   
1  2.619246 -0.765884 -0.093780  ... -0.819750  0.012037  2.038836  0.468579   
2 -4.219054 -1.184919 -1.240310  ... -0.604501  0.750054 -3.360521  0.856988   
3  4.499666  1.038741 -1.092716  ...  1.022959  1.275598 -3.480110 -1.065252   
4 -4.290282 -1.761308  0.807652  ...  0.513906 -1.803473  0.518579 -0.205029   

         34        35        36        37        38        39  
0  3

In [25]:
labeled_y = y.copy()
labeled_y.columns = ['Result']
full = pd.concat([X, labeled_y], axis=1)
print(f"Original shape: {full.shape}")

# https://www.scribbr.com/statistics/outliers/
# Remove outliers
for col in full.columns:
    if col == 'Result':
        continue
    q1 = full[col].quantile(0.25)
    q3 = full[col].quantile(0.75)
    iqr = q3 - q1
    threshold = 1.5 * iqr
    # Remove rows that are outliers
    full = full[(full[col] >= q1 - threshold) & (full[col] <= q3 + threshold)] 
    
print(f"Shape after removing outliers: {full.shape}")

new_y = full['Result']
new_X = full.drop('Result', axis=1)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(
    new_X, new_y,
    train_size=0.8, test_size=0.2,
    random_state=0
)


Original shape: (1000, 41)
Shape after removing outliers: (747, 41)


In [28]:
# Find hyperparameters with grid search
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'classifier__n_estimators': range(100, 550, 100),
    'classifier__max_depth': range(1, 20, 10),
    'classifier__min_samples_leaf': range(1, 4, 1),
}

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=0))
])

# Create a GridSearchCV object
# grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Fit the GridSearchCV object to the data
grid_search.fit(X_train_full, y_train.values.ravel())

# Print the best hyperparameters and the corresponding mean cross-validated score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# scores = {}
# for n_estimators in range(100, 550, 50):
#     model = RandomForestClassifier(n_estimators, random_state=0)
#     piepline = Pipeline(steps=[
#         ('model', model)
#     ])
#     mean_score = -1 * cross_val_score(
#         piepline, X, y.values.ravel(), cv=5, scoring='neg_mean_absolute_error').mean()
#     scores[n_estimators] = mean_score
#     print(f"n_estimators: {n_estimators}, mean_score: {mean_score}")

# print(scores)


Best hyperparameters:  {'classifier__max_depth': 11, 'classifier__min_samples_leaf': 1, 'classifier__n_estimators': 400}
Best score:  0.8609803921568627


In [29]:

# Break off validation set from training data
# X_train_full, X_valid_full, y_train, y_valid = train_test_split(
#     X, y,
#     train_size=0.8, test_size=0.2,
#     random_state=0
# )

scaler = StandardScaler()
model = RandomForestClassifier(n_estimators=400, max_depth=11, min_samples_leaf=2, random_state=0)

piepline = Pipeline(steps=[
    ('scaler', scaler),
    ('model', model)
])
piepline.fit(X_train_full, y_train.values.ravel())
cmatrix = confusion_matrix(y_valid, model.predict(X_valid_full))
print(cmatrix)


[[66  7]
 [14 63]]
